In [1]:
# import standard PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support


# import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms


The following library call downloads the training set and puts it into data/FashionMNIST, and prepares the dataset to be passed into a pytorch as a tensor.

In [2]:
# Use standard FashionMNIST dataset
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

test_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = False,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)


Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw




Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw




Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw




Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [13]:
# Build the neural network, expand on top of nn.Module
class Network(nn.Module):
  def __init__(self):
    super().__init__()

    # define layers
    # self.fc1 = nn.Linear(in_features=28*28,out_features=200)
    # self.fc2 = nn.Linear(in_features=200,out_features=10)
    
    # todo add padding, dilation
    self.p = 2
    self.k = 5
    self.c1 = 10
    self.c2 = 20
    self.ts = 28
    
    #size of torch after the first convolution and maxpooling
    self.ts = int((1+self.ts-self.k+2*self.p)/2)
    #size of torch after the second convolution and maxpooling
    self.ts = int((1+self.ts-self.k+2*self.p)/2)

    self.cl1 = nn.Conv2d(1, self.c1, kernel_size = (self.k,self.k), padding = self.p)
    self.bn1 = nn.BatchNorm2d(self.c1)
    self.mp1 = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2))
    
    self.cl2 = nn.Conv2d(self.c1,self.c2,kernel_size = (self.k,self.k), padding =self.p)
    self.bn2 = nn.BatchNorm2d(self.c2)
    self.mp2 = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2))
    
    self.fc3 = nn.Linear(in_features=(self.ts*self.ts*self.c2),out_features=120)
    self.fc4 = nn.Linear(in_features=120,out_features = 60)
    self.fc5 = nn.Linear(in_features=60,out_features = 10)

    self.do = nn.Dropout2d(0.5)
    


  # define forward function
  def forward(self, t):
    # fc 1
    # t=t.reshape(-1,28*28)
    # t=self.fc1(t)
    # t=F.relu(t)

    # fc 2
    # t=self.fc2(t)
    # don't need softmax here since we'll use cross-entropy as activation.

    #cl1
    t = self.cl1(t)
    t = self.bn1(t)
    # print("cl1 output shape:", t.shape)

    t = F.relu(t)
    t = self.mp1(t)
    # print("cl1 mp output shape:", t.shape)
    #cl2 
    t = self.cl2(t)
    t = self.bn2(t)
    t = F.relu(t)
    t = self.mp2(t)
    # print("cl2 output shape:", t.shape)
    
    #drop out
    t = self.do(t)
    #layer 3
    #fully connected 
    t = t.reshape(-1,self.ts*self.ts*self.c2)
    t= self.fc3(t)
    t = F.relu(t)
    
    # print("fc1 output shape:", t.shape)

    t = self.fc4(t)
    t = F.relu(t)
    # print("fc2 output shape:", t.shape)

  
    t = self.fc5(t)
    # t = F.relu(t)
    # t= self.fc6(t)
    # print("output shape:", t.shape)


    return t

#Accuracy 
Auxiliary function that reports the accuracy on a dataset.

In [6]:
def get_accuracy(model,dataloader):
  count=0
  correct=0

  model.eval()
  with torch.no_grad():
    for batch in dataloader:
      images = batch[0]
      labels = batch[1]
      preds=model(images)
      batch_correct=preds.argmax(dim=1).eq(labels).sum().item()
      batch_count=len(batch[0])
      count+=batch_count
      correct+=batch_correct
  model.train()
  return correct/count


Train the model, report the training set accuracy after each epoch.

In [14]:
lr=0.001
batch_size=1000
shuffle=True
epochs=10

network = Network()
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

# set the network to training mode
network.train()
for epoch in range(epochs):
  for batch in loader:
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch {0}: train set accuracy {1}'.format(epoch,get_accuracy(network,loader)))

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)
print('Epoch {0}: test set accuracy {1}'.format(epoch,get_accuracy(network,test_loader)))




size :  7
Epoch 0: train set accuracy 0.8501833333333333
Epoch 1: train set accuracy 0.8754
Epoch 2: train set accuracy 0.8853666666666666
Epoch 3: train set accuracy 0.89315
Epoch 4: train set accuracy 0.9007666666666667
Epoch 5: train set accuracy 0.9032166666666667
Epoch 6: train set accuracy 0.9044
Epoch 7: train set accuracy 0.9079666666666667
Epoch 8: train set accuracy 0.90955
Epoch 9: train set accuracy 0.9146166666666666
Epoch 9: test set accuracy 0.895
